In [2]:
import numpy as np # Library for vectorized data
import pandas as pd # Library for data analysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
import json # Library for JSON files
from geopy.geocoders import Nominatim # Converting an address into Latitude/Longitude values
import requests # Library for requests of data
from bs4 import BeautifulSoup # library to parse out HTML and XML documents
from pandas.io.json import json_normalize # Tranforming a JSON file into a pandas dataframe
import matplotlib.cm as cm # Matplotlib and associated plotting modules
import matplotlib.colors as colors
from sklearn.cluster import KMeans # importing k-means from clustering stage
!pip install folium
import folium # Library for map-rendering
print("Libraries imported.")

     |████████████████████████████████| 102kB 10.0MB/s ta 0:00:01
Libraries imported.


In [3]:
TorontoData = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(TorontoData, 'html.parser')
PostalCodeList = []
BoroughList = []
NeighborhoodList = []

In [4]:
# Sorting data into their respective lists
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        PostalCodeList.append(cells[0].text)
        BoroughList.append(cells[1].text)
        NeighborhoodList.append(cells[2].text.rstrip('\n')) # Avoiding any new lines in neighborhood cell

In [5]:
# New Dataframe
Toronto_df = pd.DataFrame({"PostalCode": PostalCodeList, "Borough": BoroughList, "Neighborhood": NeighborhoodList})
Toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned
1,M2A\n,Not assigned\n,Not assigned
2,M3A\n,North York\n,Parkwoods
3,M4A\n,North York\n,Victoria Village
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront"


In [6]:
# Get rid of 'Not assigned' boroughs
Toronto_df_DropNA = Toronto_df[Toronto_df.Borough != "Not assigned\n"].reset_index(drop=True)
Toronto_df_DropNA.head()

,PostalCode,Borough,Neighborhood
0,M3A\n,North York\n,Parkwoods
1,M4A\n,North York\n,Victoria Village
2,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront"
3,M6A\n,North York\n,"Lawrence Manor, Lawrence Heights"
4,M7A\n,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government"


In [7]:
# Grouping neighborhoods that are in the same borough
Toronto_df_Grouped = Toronto_df_DropNA.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
Toronto_df_Grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B\n,Scarborough\n,"Malvern, Rouge"
1,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek"
2,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill"
3,M1G\n,Scarborough\n,Woburn
4,M1H\n,Scarborough\n,Cedarbrae


In [8]:
# For Neighborhoods that are "Not assigned", make the value the same as Borough
for index, row in Toronto_df_Grouped.iterrows():
    if row["Neighborhood"] == "Not assigned\n":
        row["Neighborhood"] = row["Borough"]
Toronto_df_Grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B\n,Scarborough\n,"Malvern, Rouge"
1,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek"
2,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill"
3,M1G\n,Scarborough\n,Woburn
4,M1H\n,Scarborough\n,Cedarbrae


In [9]:
# Test dataframe, just to check it is all done to the requirements of the assessment.
column_names = ["PostalCode", "Borough", "Neighborhood"]
Test_df = pd.DataFrame(columns=column_names)
Test_list = ["M5G\n", "M2H\n", "M4B\n", "M1J\n", "M4G\n", "M4M\n", "M1R\n", "M9V\n", "M9L\n", "M5V\n", "M1B\n", "M5A\n"]
for postcode in Test_list:
    Test_df = Test_df.append(Toronto_df_Grouped[Toronto_df_Grouped["PostalCode"]==postcode], ignore_index=True)
Test_df

,PostalCode,Borough,Neighborhood
0,M5G\n,Downtown Toronto\n,Central Bay Street
1,M2H\n,North York\n,Hillcrest Village
2,M4B\n,East York\n,"Parkview Hill, Woodbine Gardens"
3,M1J\n,Scarborough\n,Scarborough Village
4,M4G\n,East York\n,Leaside
5,M4M\n,East Toronto\n,Studio District
6,M1R\n,Scarborough\n,"Wexford, Maryvale"
7,M9V\n,Etobicoke\n,"South Steeles, Silverstone, Humbergate, Jamest..."
8,M9L\n,North York\n,Humber Summit
9,M5V\n,Downtown Toronto\n,"CN Tower, King and Spadina, Railway Lands, Har..."


In [10]:
# Number of rows of the cleaned dataframe
Toronto_df_Grouped.shape

(103, 3)

In [21]:
#Removing useless characters from 'PostalCode' & 'Borough' columns.
Toronto_df_Grouped['PostalCode']=Toronto_df_Grouped['PostalCode'].str.replace('\n','')
Toronto_df_Grouped['Borough']=Toronto_df_Grouped['Borough'].str.replace('\n','')
Toronto_df_Grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [22]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [23]:
# Renaming the column "PostalCode"
df_data_1.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
df_data_1.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [24]:
# Merging two tables on the column "PostalCode"
Toronto_df_new = Toronto_df_Grouped.merge(df_data_1, on="PostalCode", how="left")
Toronto_df_new.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [25]:
# Creating a new test dataframe
column_names = ["PostalCode", "Borough", "Neighborhood", "Latitude", "Longitude"]
test_df_2 = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df_2 = test_df_2.append(Toronto_df_new[Toronto_df_new["PostalCode"]==postcode], ignore_index=True)
    
test_df_2

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,"Wexford, Maryvale",43.750072,-79.295849
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420
